# Ozone: MERGED-UV

# Import packages¶

In [ ]:
import calendar

import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

# Define parameters

In [ ]:
# Time
year_start = 1996
year_stop = 2022

# Regions
region_slices = {
    "global": {
        "lat_slice": slice(-90, 90),
        "lon_slice": slice(0, 360),
    },
    "tropics": {
        "lat_slice": slice(-25, 25),
        "lon_slice": slice(0, 360),
    },
    "NH mid-latitudes": {
        "lat_slice": slice(30, 60),
        "lon_slice": slice(0, 360),
    },
    "SH mid-latitudes": {
        "lat_slice": slice(-30, -60),
        "lon_slice": slice(0, 360),
    },
    "NH polar": {
        "lat_slice": slice(60, 90),
        "lon_slice": slice(0, 360),
    },
    "SH polar": {
        "lat_slice": slice(-60, -90),
        "lon_slice": slice(0, 360),
    },
}

# Variable to show
varname = "total_ozone_column"

# Define request

In [ ]:
collection_id = "satellite-ozone-v1"
requests = {
    "format": "zip",
    "processing_level": "level_3",
    "variable": "atmosphere_mole_content_of_ozone",
    "sensor": "merged_uv",
    "vertical_aggregation": "total_column",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{month:02d}" for month in range(1, 12 + 1)],
    "version": "v2000",
}

# Define transform function:

In [ ]:
def convert_source_to_time(ds):
    # Naming convention: YYYYMM-*.nc
    ds["source"] = pd.to_datetime(ds["source"].str.slice(None, 6), format="%Y%m")
    return ds.rename(source="time")


def spatial_weighted_mean(ds, lon_slice, lat_slice):
    ds = convert_source_to_time(ds)
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    return diagnostics.spatial_weighted_mean(ds)


def time_weighted_mean(ds):
    ds = convert_source_to_time(ds)
    return diagnostics.time_weighted_mean(ds)

# Download and transform data

In [ ]:
kwargs = {"collection_id": collection_id, "requests": requests, "chunks": {"year": 1}}

# Timeseries
datasets = []
for region, transform_func_kwargs in region_slices.items():
    print(f"{region=}")
    ds = download.download_and_transform(
        **kwargs,
        transform_func=spatial_weighted_mean,
        transform_func_kwargs=transform_func_kwargs,
        drop_variables="time_bounds",
    )
    datasets.append(ds.expand_dims(region=[region]))
ds_timeseries = xr.concat(datasets, "region")

# Original data
ds = download.download_and_transform(
    **kwargs,
    transform_func=convert_source_to_time,
)

## Plot Data Coverage

In [ ]:
da_obs = ds[f"{varname}_number_of_observations"]
plot.projected_map(da_obs.sum(dim="time", keep_attrs=True), show_stats=False)

fig, axs = plt.subplots(2, 1)
for ax, dim in zip(axs.flatten(), ("longitude", "latitude")):
    plot_obj = da_obs.sum(dim, keep_attrs=True).plot.contourf(
        x="time",
        ax=ax,
        levels=range(0, 10_500 + 1, 1_500),
        extend="max",
        add_colorbar=False,
    )
label = da_obs.attrs["long_name"]
isplit = len(label) // 2
label = "\n".join([label[:isplit], label[isplit:]])
_ = fig.colorbar(plot_obj, ax=axs.flatten(), label=label)

## Map of the Total Ozone column

In [ ]:
da = ds[varname]
with xr.set_options(keep_attrs=True):
    da = da * da.attrs["multiplication_factor_to_convert_to_DU"]
da.attrs["units"] = "DU"

contour_kwargs = {"levels": range(160, 460, 10), "extend": "both", "cmap": "RdGy_r"}
map = plot.projected_map(
    da.mean("time", keep_attrs=True),
    plot_func="contourf",
    show_stats=False,
    **contour_kwargs,
)

## Annual Cycle

In [ ]:
da_annual_cycle = da.groupby("time.month").mean(["time", "longitude"], keep_attrs=True)
da_annual_cycle["month"] = [
    calendar.month_abbr[m] for m in da_annual_cycle["month"].values
]
_ = da_annual_cycle.plot.contourf(x="month", **contour_kwargs)

## Time series

In [ ]:
with xr.set_options(keep_attrs=True):
    da_anomaly = (
        ds_timeseries[varname].groupby("time.month")
        - ds_timeseries[varname].groupby("time.month").mean()
    )
    da_anomaly *= da_anomaly.attrs["multiplication_factor_to_convert_to_DU"]
da_anomaly.attrs["units"] = "DU"
facet = da_anomaly.plot(row="region", col_wrap=2)
for ax in facet.axs.flatten():
    ax.grid()
facet.fig.autofmt_xdate(rotation=45)